In [1]:
import random
import mne, os, glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib qt 

In [2]:
""" 
Loading data
"""

raw_data_dir = '/Users/mvmigem/Documents/data/project_1/raw_data/'
example_data_dir = '/Users/mvmigem/Documents/data/project_1/preprocessed/average_ref/'
cleaned_data_dir = '/Users/mvmigem/Documents/data/project_1/preprocessed/mastoid_ref/'
# Indicate which subjects are being done
n_subs = 26
subj_base = np.arange(2,n_subs+1) # list of participants that are going to be processed


In [3]:
sub = 26
# Load behavioural data
behav_path = os.path.join(raw_data_dir,f'sub_{sub}/behav/predatt_participant_{sub}.csv')
behav_data = pd.read_csv(behav_path)
# Load cleaned epoch file
epoch_path = os.path.join(cleaned_data_dir,f'unpaired/main_clean_mastoidref_{sub:02}-epo.fif')
epoch = mne.read_epochs(epoch_path)
ep_events = epoch.events
# Add the metadata and add a column for later regular trial subselection
behav_data['precedes_odd'] = behav_data['expected'].shift(-1).eq('odd').astype(int)
meta_data = pd.DataFrame(np.repeat(behav_data.values, 5, axis=0), columns=behav_data.columns)
# epoch.metadata = meta_data

Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\unpaired\main_clean_mastoidref_26-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
4197 matching events found
No baseline correction applied
0 projection items activated


In [4]:
check_missing = []
for i in range(len(ep_events)-5):
    if ep_events[i,2] == 99:
        if  ep_events[i+5,2] != 99:
            check_missing.append(i+5)

In [4]:
# meta_data = meta_data.iloc[:-5]
meta_data = meta_data.iloc[:-1] # pp1

In [5]:
meta_data = meta_data.iloc[1:]

In [5]:

# there was an error that accidentaly dropped the last/first stims of  trials (prob due to block cuttoff)
lost_stim = []
shift_comp = 0
for i in range(len(ep_events)-5):
    if ep_events[i,2] == 99:
        if ep_events[i +5,2] !=99:
            lost_stim.append(i+5+ shift_comp)
            shift_comp += 1
meta_data = meta_data.drop(lost_stim).reset_index(drop=True)


In [6]:
epoch.metadata = meta_data
epoch_save_path = os.path.join(cleaned_data_dir,f'main_eventset_mastoidref_{sub:02}-epo.fif')
epoch.save(epoch_save_path)

Adding metadata with 33 columns


In [ ]:
for subj in range(1,22):
    if subj == 20:
        continue
    epoch_path = os.path.join(cleaned_data_dir,f'unpaired/main_clean_mastoidref_{subj:02}-epo.fif')
    epoch = mne.read_epochs(epoch_path)
    ep_events = epoch.events
    epoch_exmp_path = os.path.join(example_data_dir,f'main_eventset_averageref_{subj:02}-epo.fif')
    epoch_exmp = mne.read_epochs(epoch_exmp_path)
    meta_data = epoch_exmp.metadata

    epoch.metadata = meta_data
    epoch_save_path = os.path.join(cleaned_data_dir,f'main_eventset_mastoidref_{subj:02}-epo.fif')
    epoch.save(epoch_save_path)